In [1]:
# Gradio ve pandas kütüphanelerini yükle
!pip install gradio pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import gradio as gr  # Gradio, web tabanlı uygulama oluşturmak için
import pandas as pd  # Pandas, veri işleme ve analizi için
import re  # Regex işlemleri için

In [3]:
# Excel dosyasını okuma
excel_dosyasi = "/content/argo_ve_yoresel_kelimeler.xlsx"
veri = pd.read_excel(excel_dosyasi)

# Kelime, kategori, anlam ve standart karşılık sütunlarını kontrol etme
print(veri.head())  # İlk birkaç satırı kontrol etmek

        Kelime Kategori                  Anlam         Bölge Standart Karşılık
0        Kanka     Argo          Arkadaş, dost         Genel           Arkadaş
1          Aga  Yöresel     Büyük kişi, ağabey  Doğu Anadolu            Ağabey
2        Çomar     Argo  Kaba, terbiyesiz kişi         Genel         Kaba kişi
3  Mıncıklamak     Argo       Sıkmak, uğraşmak         Genel            Sıkmak
4        Çıtır     Argo       Genç, güzel kişi         Genel              Genç


In [4]:
# Metindeki argo ve yöresel kelimeleri tespit eden bir fonksiyon oluşturma
def argo_yoresel_tespit(metin):
    bulunan_kelimeler = []
    temiz_metin = metin

    # Veriyi kontrol ederek kelimeleri tespit etme
    for index, row in veri.iterrows():
        kelime = row['Kelime']

        # Regex ile kelimeyi ekleriyle birlikte kontrol edecek bir desen oluşturma

        pattern = r'\b' + re.escape(kelime) + r'(\w*?)\b'
        # Eğer kelime metinde bulunuyorsa (büyük küçük harf duyarsız)

        if re.search(pattern, metin, re.IGNORECASE):
            bulunan_kelimeler.append({
                "Kelime": kelime,
                "Kategori": row['Kategori'],
                "Anlam": row['Anlam'],
                "Standart Karşılık": row['Standart Karşılık']
            })
            # Kelimeyi standart karşılık ile değiştir
            temiz_metin = re.sub(pattern, row['Standart Karşılık'], temiz_metin, flags=re.IGNORECASE)
    # Temizlenmiş metni ve bulunan kelimelerin listesini döndürme
    return temiz_metin, bulunan_kelimeler

In [5]:
# Gradio arayüzü tanımlama
def arayuz_analiz(metin):
    temiz_metin, bulunan_kelimeler = argo_yoresel_tespit(metin) # Tespit fonksiyonunu çağır

    # Eğer bulunan kelimeler varsa, her kelimenin bilgilerini listele
    if bulunan_kelimeler:
        kelime_bilgisi = "\n\n".join([
            f"Kelime: {kelime['Kelime']}, \nKategori: {kelime['Kategori']}, \nAnlam: {kelime['Anlam']}, \nStandart Karşılık: {kelime['Standart Karşılık']}"
            for kelime in bulunan_kelimeler
        ])
    else:
        kelime_bilgisi = "Metinde argo veya yöresel kelime bulunamadı." # Eğer kelime bulunmazsa bir mesaj döndür

    # Temiz metni ve bulunan kelimeleri geri döndür
    return temiz_metin, kelime_bilgisi

In [6]:
# Gradio arayüzü tanımlama
gradio_arayuz = gr.Interface(
    fn=arayuz_analiz,  # Kullanıcıdan metin alındığında çalışacak fonksiyon
    inputs=gr.Textbox(label="Metin Girin"),
    outputs=[
        gr.Textbox(label="Temizlenmiş Metin"),
        gr.Textbox(label="Bulunan Kelime Bilgileri")
    ],
    title="Argo ve Yöresel Kelime Tespiti",
    description="Metin içindeki argo ve yöresel kelimeleri tespit edip anlamlarını ve standart karşılıklarını gösterir."
)

In [7]:
# Arayüzü başlat
gradio_arayuz.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cb9db89e7a5da63df1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
